In [1]:
import os
import numpy as np
import torch
import torch_geometric.data

import meld_graph
import meld_graph.training
import meld_graph.models
import meld_graph.experiment
import meld_graph.dataset
from meld_graph.paths import load_config
from meld_graph.paths import EXPERIMENT_PATH
from meld_graph.dataset import GraphDataset, Oversampler

Setting MELD_DATA_PATH to /rds/project/kw350/rds-kw350-meld/meld_data/Data
Setting BASE_PATH to /rds/project/kw350/rds-kw350-meld/meld_data/Data
Setting EXPERIMENT_PATH to /rds/project/kw350/rds-kw350-meld/experiments/co-ripa1/
Setting FS_SUBJECTS_PATH to 
Setting EXPERIMENT_PATH to /rds/project/kw350/rds-kw350-meld/experiments_graph/co-ripa1


In [33]:
# create experiment
config_file = '/rds/user/co-ripa1/hpc-work/scripts/meld_classifier_GDL/scripts/config_files/base_config_test.py'
config = load_config(config_file)
exp = meld_graph.experiment.Experiment(config.network_parameters, config.data_parameters)


Saving parameter files to /rds/project/kw350/rds-kw350-meld/experiments_graph/co-ripa1/23-05-19_MATH_test_histology_head/fold_00
Initialised Experiment 23-05-19_MATH_test_histology_head


In [41]:
import importlib
importlib.reload(meld_graph.dataset)
importlib.reload(meld_graph.data_preprocessing)
importlib.reload(meld_graph.training)

<module 'meld_graph.training' from '/rds/user/co-ripa1/hpc-work/scripts/meld_classifier_GDL/meld_graph/training.py'>

In [42]:
# create the trainer the model
trainer = meld_graph.training.Trainer(exp)
trainer.experiment.load_model()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
trainer.experiment.model.to(device)
# trainer.train()


Model already exists. Specify force=True to force reloading and initialisation
Creating model


In [43]:
train_dset = GraphDataset.from_experiment(trainer.experiment, mode='train')
print(train_dset)


Loading and preprocessing train data
Z-scoring data for MELD_H5_3T_FCD_0002
Z-scoring data for MELD_H5_3T_C_0005
Z-scoring data for MELD_H5_3T_C_0026
Z-scoring data for MELD_H5_3T_FCD_0031
Z-scoring data for MELD_H5_3T_FCD_0016
Z-scoring data for MELD_H5_3T_FCD_0023
Z-scoring data for MELD_H5_3T_C_0023
Z-scoring data for MELD_H5_3T_C_0020
Z-scoring data for MELD_H5_3T_C_0008
Z-scoring data for MELD_H5_3T_C_0022
Z-scoring data for MELD_H5_3T_FCD_0010
Z-scoring data for MELD_H5_3T_C_0028
Z-scoring data for MELD_H5_3T_C_0024
Z-scoring data for MELD_H5_3T_FCD_0030
Z-scoring data for MELD_H5_3T_FCD_0003
Z-scoring data for MELD_H5_3T_C_0013


GraphDataset(32)


In [44]:
val_dset = GraphDataset.from_experiment(trainer.experiment, mode='val')
print(val_dset)

Loading and preprocessing val data
Z-scoring data for MELD_H5_3T_C_0003
Z-scoring data for MELD_H5_3T_FCD_0019
Z-scoring data for MELD_H5_3T_FCD_0008


GraphDataset(6)


In [45]:
sampler=None
shuffle = trainer.params['shuffle_each_epoch']
batch_size = trainer.params['batch_size']
optimiser = torch.optim.SGD(trainer.experiment.model.parameters(), **trainer.params['optimiser_parameters'])
optimiser = optimiser

train_data_loader = torch_geometric.loader.DataLoader(
             train_dset, sampler=sampler, 
             shuffle=shuffle,
             batch_size= batch_size,
             num_workers=0, persistent_workers=False, prefetch_factor=2
             )

In [47]:
cur_scores = trainer.train_epoch(train_data_loader, optimiser)

In [48]:
cur_scores

{'cross_entropy': 0.7883511781692505,
 'dice': 0.9784323046915233,
 'distance_regression': 0.28809136897325516,
 'lesion_classification': 0.6396960318088531,
 'histology_classification': 1.3424707651138306,
 'ds6_cross_entropy': 0.008665163069963455,
 'ds5_cross_entropy': 0.02071181684732437,
 'ds4_cross_entropy': 0.041922105476260185,
 'ds3_cross_entropy': 0.06115961913019419,
 'ds2_cross_entropy': 0.1806270331144333,
 'ds1_cross_entropy': 0.34350793808698654,
 'ds6_dice': 0.014627989630064462,
 'ds5_dice': 0.03026165576011408,
 'ds4_dice': 0.05998166825156659,
 'ds3_dice': 0.11645015981048346,
 'ds2_dice': 0.22800071304664016,
 'ds1_dice': 0.4208209812641144,
 'ds6_distance_regression': 0.004387676890473813,
 'ds5_distance_regression': 0.011066023958846927,
 'ds4_distance_regression': 0.02937454916536808,
 'ds3_distance_regression': 0.04111327417194843,
 'ds2_distance_regression': 0.15398433804512024,
 'ds1_distance_regression': 0.27838488668203354,
 'ds6_lesion_classification': nan,

In [49]:
trainer.train(wandb_logging=False)

Model already exists. Specify force=True to force reloading and initialisation
Creating model
Loading and preprocessing train data
Z-scoring data for MELD_H5_3T_FCD_0002
Z-scoring data for MELD_H5_3T_C_0005
Z-scoring data for MELD_H5_3T_C_0026
Z-scoring data for MELD_H5_3T_FCD_0031
Z-scoring data for MELD_H5_3T_FCD_0016
Z-scoring data for MELD_H5_3T_FCD_0023
Z-scoring data for MELD_H5_3T_C_0023
Z-scoring data for MELD_H5_3T_C_0020
Z-scoring data for MELD_H5_3T_C_0008
Z-scoring data for MELD_H5_3T_C_0022
Z-scoring data for MELD_H5_3T_FCD_0010
Z-scoring data for MELD_H5_3T_C_0028
Z-scoring data for MELD_H5_3T_C_0024
Z-scoring data for MELD_H5_3T_FCD_0030
Z-scoring data for MELD_H5_3T_FCD_0003
Z-scoring data for MELD_H5_3T_C_0013
Loading and preprocessing val data
Z-scoring data for MELD_H5_3T_C_0003
Z-scoring data for MELD_H5_3T_FCD_0019
Z-scoring data for MELD_H5_3T_FCD_0008
Stopping metric set to loss 
using max_epochs 100 for lr decay
Epoch 0 :: learning rate 0.0001
/home/co-ripa1/.co

KeyboardInterrupt: 